In [2]:
import json

In [3]:
with open('primary_dataset_new.json') as f:
    data = json.loads(f.read())

FileNotFoundError: [Errno 2] No such file or directory: 'primary_dataset_new.json'

In [4]:
data = data['data']

NameError: name 'data' is not defined

In [9]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
    import requests
    from requests import utils

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if entities:
        entity = entities.get(wikidata_id)
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None  

In [19]:
data_new = []
done = 0
for ent in tqdm(data):
    ent['en_link'] = get_wikipedia_url_from_wikidata_id(ent['wd_id'])
    data_new.append(ent)
    done+=1

100%|██████████| 6527/6527 [1:00:41<00:00,  1.79it/s]


In [20]:
print(done)
print(data_new[:5])

6527
[{'en_wikipedia_title': 'Shane Warne', 'hi_wikipedia_title': 'शेन वॉर्न', 'wd_id': 'Q555240', 'en_link': 'https://en.wikipedia.org/wiki/Shane_Warne'}, {'en_wikipedia_title': 'Herodotus', 'hi_wikipedia_title': 'हिरोडोटस', 'wd_id': 'Q26825', 'en_link': 'https://en.wikipedia.org/wiki/Herodotus'}, {'en_wikipedia_title': 'Shibu Soren', 'hi_wikipedia_title': 'शिबू सोरेन', 'wd_id': 'Q3630358', 'en_link': 'https://en.wikipedia.org/wiki/Shibu_Soren'}, {'en_wikipedia_title': 'D. K. Pattammal', 'hi_wikipedia_title': 'दमाल कृष्णास्वामी पट्टम्माल', 'wd_id': 'Q464887', 'en_link': 'https://en.wikipedia.org/wiki/D._K._Pattammal'}, {'en_wikipedia_title': 'Do-Aklin', 'hi_wikipedia_title': 'गेंगनीहेस्सू', 'wd_id': 'Q104548', 'en_link': 'https://en.wikipedia.org/wiki/Do-Aklin'}]


In [22]:
with open('primary_dataset_new_withlinks.json',"w+") as f:
    json.dump(data_new , f)